In [14]:
import glob
import os
import sys
import numpy as np
import pdb
import shutil
from pprint import pprint # for printing dictionaries
import utils

train_dir = '/home/disk2/train/'
val_dir = '/home/disk2/val/'
test_dir = '/home/data/test/'

new_val_dir = '/home/data/val_balance/'
new_test_dir = '/home/data/test_balance/'

train_len = len(os.listdir(train_dir))
val_len = len(os.listdir(val_dir))
test_len = len(os.listdir(test_dir))
print(train_len, val_len, test_len)

# train_dir = '/home/data/train/'
# val_dir = '/home/disk2/val/'
# test_dir = '/home/data/test/'
# transfer_dir = '/scratch/users/gmachi/codex/transfer/'
# model_dir = '/home/disk2/model/'


labels_dict = {"005": ("test", 0),
            "006": ("test", 1),
            "017": ("test", 1),
            "019": ("test", 1),
            "011": ("val", 0),
            "016": ("val", 1),
            "030": ("val", 1),
            "023": ("val", 1),
            "004": ("train", 0),
            "015": ("train", 0),
            "014": ("train", 1),
            "024": ("train", 1),
            "020": ("train", 1),
            "007": ("train", 1),
            "008": ("train", 1),
            "027": ("train", 1),
            "034": ("train", 1),
            "012": ("train", 1)} 

13069 4449 4507


In [19]:
def files_to_move(dir):
    all_files = os.listdir(dir)  
    negatives = []
    positives = []
    
    for i, x in enumerate(all_files):

        reg = x.split("_")[0].split("reg")[1]
        patch = x.split("_")[1].split("patch")[1]
        
#         # remove extraneous dashes
#         if "-" in patch:
#             patch = patch[1:]
                    
        if labels_dict[reg][1] == 0: # use all ~900 neg's
            negatives.append(x)
        else:
            positives.append(x)
    
    positives = list(np.random.choice(positives, len(negatives)*2, replace=False)) # downsample: 3600->1800 pos's
    to_move = positives + negatives
                    
    return to_move
        

def count_files(l):
    return len([1 for x in l])

def unique_files(l):
    return set([x.split("_")[0].split("reg")[1] for x in l])

def set_splits(l):
    uniq = set(l)
    print(len(uniq))
    all_files = [x.split("_")[0].split("reg")[1] for x in l]
    labels = [labels_dict[u][1] for u in all_files]
    pos = np.sum(labels)
    neg = len(labels) - pos
    return pos, neg  

In [54]:
# print("\ndict of augmentations...\n------------------------")
# print("train augmentations:")
# pprint(augmentations_per_reg(train_dir))
# print("val augmentations:")
# pprint(augmentations_per_reg(val_dir))
# print("test augmentations:")
# pprint(augmentations_per_reg(test_dir))


In [16]:
# create list of files to use for train
#---------------------------------------

val_to_move = files_to_move(val_dir)
test_to_move = files_to_move(test_dir)

print(len(val_to_move))
print(len(test_to_move))


2625
2646


In [20]:

# checking summary stats for this migrated set
#---------------------------------------------

print("After augmentation/up-sampling, we have...\n------------------------------------------")
print("val set size:", count_files(val_to_move))
print("test set size:", count_files(test_to_move))

print("\nSee composition of patients in sets...\n--------------------------------------")
print("val set unique files:", unique_files(val_to_move))
print("test set unique files:", unique_files(test_to_move))

print("\n(+/-) splits in sets...\n-----------------------")
print("val set split:", set_splits(val_to_move))
print("test set split:", set_splits(test_to_move))


After augmentation/up-sampling, we have...
------------------------------------------
val set size: 2625
test set size: 2646

See composition of patients in sets...
--------------------------------------
val set unique files: {'030', '016', '011', '023'}
test set unique files: {'005', '019', '006', '017'}

(+/-) splits in sets...
-----------------------
2625
val set split: (1750, 875)
2646
test set split: (1764, 882)


In [22]:
# execute migration
#------------------
    
# val_dir = '/scratch/users/gmachi/codex/data/val/'
# new_val_dir = '/scratch/users/gmachi/codex/data_balance/val_balance/'
from tqdm import tqdm

for f in tqdm(val_to_move):
    shutil.copy(val_dir + f, new_val_dir + f) # COPY not move!!!
    
# test_dir = '/scratch/users/gmachi/codex/data/test/'
# new_test_dir = '/scratch/users/gmachi/codex/data_balance/test_balance/'

for f in tqdm(test_to_move):
    shutil.copy(test_dir + f, new_test_dir + f) # COPY not move!!!
    
    
    

100%|██████████| 2646/2646 [06:27<00:00,  9.32it/s]
